# ESSL study of CHECKER dataset using WandB monitoring

<a href="https://colab.research.google.com/github/Hackathorn/ESSL-WandB/blob/master/ESSL_Study_CHECKER_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup ESSL repro in Colab

In [1]:
# ############# Setup ESSL in Colab
# %%capture

# !git clone https://github.com/Hackathorn/ESSL-WandB/
# %cd ESSL-WandB

# !pip install umap-learn
# !pip install wandb --upgrade

## Import & Setup modules

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

# import ESSL Utils modules
from utils.sample import    create_samples_from_CHECKER

from utils.point import     create_points, \
                            add_posLowD_to_points, \
                            plot_pos2D_points

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# remove slow mirror from list of MNIST mirrors
torchvision.datasets.MNIST.mirrors = [mirror for mirror in torchvision.datasets.MNIST.mirrors
                                      if not mirror.startswith("http://yann.lecun.com")]

import wandb
wandb.login()
## Define Hyperparameters

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: richardh (use `wandb login --relogin` to force relogin)


True

In [3]:
project = "ESSL-Checker-Study"

hyperparameters = dict( 
    dataset="CHECKER", 
    n_samples=3000,
    img_size=32,
    classes=10,
    blk_size=2,
    architecture="VAE",
    epochs=5,
    batch_size=128,
    learning_rate=0.005,
    lat_dim=16,
    beta=1,
    )

## Define ESSL Pileline

In [4]:
from utils.sample import create_samples
from utils.point import create_points       #, analyze_points

def ESSL_pipeline(hyperparameters):

    # tell wandb to get started
    with wandb.init(project=project, config=hyperparameters, mode="disabled"):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config
      
      # create samples from dataset
      samples_df = create_samples(config)
      
      # create points from samples
      points_df = create_points(config, samples_df)
      


    return model
### Create Samples

### Create Samples

In [5]:
from utils.sample import create_samples_from_CHECKER #, analyze_sample

def create_samples(config):
    
    samples_df = create_samples_from_CHECKER(config.n_samples, 
                                             config.img_size, 
                                             n_classes=config.classes, 
                                             n_blk_size=config.blk_size,
    )
    # wandb.log({'sample_df': sample_df})
   
    return samples_df

### Create Points

In [6]:
from utils.point import VAE_Train_Dataset, VAE, train_VAE
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

def create_points(config, samples_df):
    
    # instantiate custom DataSet for CHECKER and DataLoader
    train_dataset = VAE_Train_Dataset(samples_df, transform=None)
    train_loader = DataLoader(dataset=train_dataset,
                            batch_size=config.batch_size, 
                            shuffle=True, num_workers=2)
    print(next(iter(train_loader))[0].shape)

    # instantiate model and print
    n_channels = 1
    model = VAE(n_channels, config.img_size, config.lat_dim).to(device)
    print(model)
    
    # define optimizer and loss criterion
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    criterion = nn.MSELoss()    # >>>> TODO versus F.mse_loss

    # train VAE model for optimal D-dim latent space
    train_VAE(config,
              model, criterion, optimizer, 
              train_loader,
              device,
              )
    

    # and test its final performance
    # test(model, test_loader)

    points_df = None
    return points_df

In [7]:
############### Execute Pipeline
model = ESSL_pipeline(hyperparameters)

torch.Size([128, 1, 32, 32])
VAE(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1, inplace=True)
    (3): Dropout2d(p=0.25, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.1, inplace=True)
    (7): Dropout2d(p=0.25, inplace=False)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.1, inplace=True)
    (11): Dropout2d(p=0.25, inplace=False)
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (13): BatchNorm2d(64, eps=1e-05, mo

AttributeError: 'tuple' object has no attribute 'size'

In [9]:
def train(model, loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (images, labels) in enumerate(loader):

            loss = train_batch(images, labels, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss, example_ct, epoch)


def train_batch(images, labels, model, optimizer, criterion):
    images, labels = images.to(device), labels.to(device)
    
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [10]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [11]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test images: {100 * correct / total}%")
        
        wandb.log({"test_accuracy": correct / total})

    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")

In [12]:
# Build, train and analyze the model with the pipeline
model = ESSL_pipeline(hyperparameters)

C:\Users\richa\anaconda3\envs\ESSL\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\richa\anaconda3\envs\ESSL\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Loss after 03072 examples: 0.369
Loss after 06272 examples: 0.225
Loss after 09472 examples: 0.190
Loss after 12640 examples: 0.119
Loss after 15840 examples: 0.120
Loss after 19040 examples: 0.151
Loss after 22240 examples: 0.086
Loss after 25408 examples: 0.049
Loss after 28608 examples: 0.048
Loss after 31808 examples: 0.041
Loss after 35008 examples: 0.044
Loss after 38176 examples: 0.046
Loss after 41376 examples: 0.035
Loss after 44576 examples: 0.107
Loss after 47776 examples: 0.060
Loss after 50944 examples: 0.039
Loss after 54144 examples: 0.033
Loss after 57344 examples: 0.040
Accuracy of the model on the 2000 test images: 98.2%


epoch,▁▁▁▃▃▃▃▅▅▅▅▆▆▆▆███
loss,█▅▄▃▃▃▂▁▁▁▁▁▁▃▂▁▁▁
test_accuracy,▁
epoch,4
loss,0.0396
test_accuracy,0.982


OSError: [WinError 1314] A required privilege is not held by the client: 'c:\\Users\\richa\\Documents\\ESSL-WandB\\model.onnx' -> 'c:\\Users\\richa\\Documents\\ESSL-WandB\\wandb\\run-20220117_090855-g8m9rxn7\\files\\model.onnx'

In [2]:
'''
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> TODO add comments
'''
# %%
# import modules

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from utils.experiment import create_experiment, \
                                    get_dict_from_arg, \
                                    save_dataframe, \
                                    load_dataframe

from utils.sample import  create_samples

from utils.point import create_points, \
                        add_posLowD_to_points, \
                        plot_pos2D_points

from utils.edge import    create_edges_with_UMAP, \
                    add_pt_degreeweights_to_points, \
                    add_Euclidean_distance_to_edges
                    
# from points import load_previous_points
# from STEP5_Subgraphs import find_subgraph_cover_from_cliques
from utils.subgraph import create_covers_from_edges, plot_covers, create_subgraph_from_covers
# from utils.mapping import create_mappings
# from STEP6_Mappings import find_peak_step_in_cover 
# from STEP6_Mappings import plot_density_of_sg_class_map


In [ ]:
# %%
def set_params():
    """set_params [summary]

    Returns:
        dict: Key/value for each parameter for process_run. If value is list,
                param is a hyper-parameter, for which each combination generates a run
    """
    
    dsn = 'MNIST'          # ['MNIST','FASHION",'DCAI','CHECKER']   # MNIST and/or DCAI 
    
    tag = 'COMBO_base' if isinstance(dsn, list) else dsn + '_base'
        
    params = {  
            ########### parameters for experiment.py - scalar values only
            'exper_tag':        # experiment name
                tag,
            'verbose': 
                1, 
            'filelog':          # print log to file
                1,
            ########### parameters for process_run
            #   Use 3-char abrevation as dict key. Key values can be scalar or list.
            #   If list, it is a hyper-parameter, which appears in Run folder name.
            #   If list len > 1, every combination results in an unique run.
            
            'dsn':              # dataset_path= MNIST, DCAI, CHECKER >>> TODO path to structured dataset folder
                dsn,
            'pts':              # points_path= ''; if not null, use path for points_df to bypass VAE training
                '', 
            'img':              # img_size= 32, 64, 96, 128; pixel width/height of square image
                32,
            'ncl':              # no of classes in samples (CHECKER)
                [10],
            'nbk':              # no of pixel blocks in image to vary info density (CHECKER)
                [1],
            'lr':               # learning rate during training
                [0.0005],
            'dim':              # latent_dim = 8, 16, 32, 64, or any value; dim = 2 & 3 auto perform
                # [8,16],
                [16],
            'epo':              # epoches= 10, 100, 200; no of epoch training cycles
                5, 
            'n_n':              # nearest_neighbor= 2 to 25% of sample size; UMAP  param 15 default
                # [2,15,100],
                15,
            'dis':              # min_dist= 0.0 to 0.99; fine clumpiest to global structure - 0.1 default
                # [0.1,0.99],
                0.1,
            'cut':              # wgt_cutoff= 0.01; ignore edges with weights below cutoff 
                0.01,
        }
    
    nRuns = np.prod([len(v) for v in params.values() if isinstance(v, list) ])
    hparams_keys = [k for k, v in params.items() if isinstance(v, list) ]
    if nRuns > 1: print(f'No of Runs = {nRuns} for hparams key = {hparams_keys}')
    
    return params 




In [ ]:
# %%
def process_run(run_folder, params):
    """process_run routine is executed by create_experiment for each run

        Args:
        run_folder (str):   path to run folder
        params (dict):      See set_params method for param key definitions

        Returns:
        [dict]: key/value for metrics from this run
        """

    ##### set param variables from params arg
    global verbose
    verbose = params['verbose'] if 'verbose' in params else True    # >>> TODO test verbose from ESSL.py

    ##### set hparam variables from param arg
    dataset_path = params['dsn'] if 'dsn' in params else None
    # preprocess = params['pre'] if 'pre' in params else True  >>> TODO remove! samples_df is responsible
    n_classes = params['ncl'] if 'ncl' in params else 10
    n_blk_size = params['nbk'] if 'nbk' in params else 2
    points_path = params['pts'] if 'pts' in params else ''
    img_size = params['img'] if 'img' in params else 32
    nEpochs = params['epo'] if 'epo' in params else 10
    latent_dim = params['dim'] if 'dim' in params else 16
    n_neighbors = params['n_n'] if 'n_n' in params else 5
    min_dist = params['dis'] if 'dis' in params else 0.1
    wgt_cutoff = params['cut'] if 'cut' in params else 0 # accept ALL as default
    pct_node_covered = params['pct_node_covered'] if 'pct_node_covered' in params else 0.9
    
    ##### STEP1 - Create samples by preprocessing initial dataset
    samples_df = create_samples(dataset_path, img_size, n_classes=n_classes, 
                                n_blk_size=n_blk_size, verbose=verbose)
    # sample_df = convert_ds_to_df(dataset_folder, img_invert=True)
    # print('preprocess_images_to_array img_size = ', img_size)
    # # sample_df = preprocess_images_to_array(sample_df, img_size)
    # sample_df = preprocess_images_to_array(sample_df, 256)      # save as many pixels as possible TODO flags!!!
    save_dataframe(run_folder, 'samples_df', samples_df)

    ##### STEP2 - Create points by embedding samples into D-dim Latent Space
    if points_path == '':   # if no previous points_df, create a new points_df
        # points_df = create_points(samples_df, img_size, preprocess, latent_dim, nEpochs, run_folder) >>> TODO remove! samples_df is responsible
        points_df = create_points(samples_df, img_size, latent_dim, nEpochs, run_folder, verbose=verbose)
        save_dataframe(run_folder, 'points_df', points_df)
    else:
        points_df = load_dataframe(points_path, 'points_df')

    points_df = add_posLowD_to_points(points_df, n_neighbors, min_dist)
    save_dataframe(run_folder, 'points_df', points_df)
    plot_pos2D_points(points_df, n_neighbors, min_dist, run_folder)

    ##### STEP3 -  Create edges by calculating UMAP weights among points >>>TODO add more distance metrics
    edges_df, _ = create_edges_with_UMAP(points_df, n_neighbors, min_dist, latent_dim, wgt_cutoff, verbose)
    edges_df = add_Euclidean_distance_to_edges(edges_df, points_df, verbose) 
    # edges_df = add_labels_to_edges(edges_df, points_df)   # TODO FLAG edge if different labels?
    save_dataframe(run_folder, 'edges_df', edges_df)

    # points_df = add_pt_degreeweights_to_points(points_df, edges_df, run_folder, verbose)
    # save_dataframe(run_folder, 'points_df', points_df)
    
    ##### STEP4 - Create Subgraphs from Edges
    # create covers by merging edges, strongest first
    covers_df = create_covers_from_edges(edges_df, verbose)
    # save_dataframe(run_folder, 'covers_df', covers_df)    # >>> TODO HUGE!!! need to save?
    # plot results of cover builds
    plot_covers(covers_df, run_folder)
    # create the subgraph based on % of points/nodes covered
    subgraphs_df = create_subgraph_from_covers(covers_df, run_folder, [70,50,40,20,10])
    save_dataframe(run_folder, 'subgraphs_df', subgraphs_df)    

    # ##### STEP4 - Create cliques among points with strong edges
    # cliques_df = find_cliques_from_edges(points_df, edges_df, umap_object, wgt_cutoff)
    # plot_cliq_weight_by_K(cliques_df, wgt_cutoff, run_folder)
    # save_dataframe(run_folder, 'cliques_df', cliques_df)

    # #### STEP4 - Create subgraphs by merging edges in Graph Space
    # covers_df = find_subgraph_cover_from_cliques(cliques_df, wgt_cutoff, run_folder)
    # save_dataframe(run_folder, 'covers_df', covers_df)
    # subgraphs_df = create_subgraphs(edges_df, pct_node_covered, run_folder)
    # save_dataframe(run_folder, 'subgraphs_df', subgraphs_df)

    ##### STEP5 - Create mappings of subgraphs to pre/post class labels
    # mappings_df = create_mappings(subgraphs_df, points_df, run_folder)   # >>>>>>>> TODO
    # save_dataframe(run_folder, 'mappings_df', mappings_df)


    ##### Create CSV/JSON for Nodes/Cliqeus to ESSL Workshop    >>>>>> TODO 
    # save_nodes_for_run(run_folder, points_df, node_lim=None)
    # save_cliques_for_run(run_folder, cliques_df, cliq_lim = None, k_lim=None)

    # save_nodes_cliques_as_CSV(run_folder, points_df, cliques_df)
    # save_nodes_as_json(run_folder, points_df, node_lim=None)
    # save_cliques_as_json(run_folder, points_df, Kcliq_lim=None)

    ####################### Update results from this runs using metrics from sample_df, etc
    ##### metrics from sample_df
    m_flagged_images = sum([1 for s in samples_df['flags'] if s != ''])

    ##### metrics from points_df
    m_flagged_points = sum([1 for s in points_df['flags'] if s != ''])
    
    mse = np.vstack(points_df.pt_mse_loss).flatten()
    m_mse_loss = mse.mean()
    z_mse = (mse - mse.mean()) / mse.std()
    mse_out = mse[z_mse > 3]
    m_mse_out_pct = 100 * len(mse_out) / len(mse)
    
    m_pos_std = np.vstack(points_df.pt_std).mean()
    # m_degree = points_df['pt_degrees'].mean()
    # m_weightsum = points_df['pt_weightsum'].mean()

    ##### metrics from edges_df
    # # m_flagged_edges = sum([1 for s in edges_df['flags'] if s != ''])  >>>>>>>>>>> TODO add 'flags' column
    # m_edge_weight = edges_df['weight'].mean()
    # m_pct_edge_weight_one = 100 * sum([1 for w in edges_df['weight'] if w == 1.0]) / len(edges_df.index)

    ##### metrics from cliques_df
    # m_flagged_cliques = sum([1 for s in cliques_df['flags'] if s != ''])  >>>>>>>>>>> TODO add 'flags' column
    # m_cliq_weight = cliques_df['weight'].mean()
    # m_cliq_ksize = cliques_df['ksize'].mean()          # >>>>>>>>>>>>>> TODO ignore 1-cliques ???
    # m_cliq_kmax = cliques_df['ksize'].max()   
    # # m_cliq_size = mean[s for s in cliques_df['ksize'] if s > 1])
    # m_pct_cliq_weight_one = 100 * sum([1 for w in cliques_df['weight'] if w == 1.0]) / len(cliques_df.index)

    ##### metrics from cover_df
    # m_flagged_covers = sum([1 for s in covers_df['flags'] if s != ''])  >>>>>>>>>>> TODO add 'flags' column
    # m_max_sg_cnt, m_max_sg_step = find_peak_step_in_cover(covers_df)

    run_metrics = { 'm_flagged_images': m_flagged_images, 
                    'm_flagged_points': m_flagged_points,
                    'm_mse_loss': m_mse_loss,
                    'm_mse_out_pct': m_mse_out_pct,
                    'm_pos_std': m_pos_std,
                    # 'm_degree': m_degree,
                    # 'm_weightsum': m_weightsum,
                    # 'm_edge_weight': m_edge_weight, 
                    # 'm_pct_edge_weight_one': m_pct_edge_weight_one,
                    # 'm_cliq_weight': m_cliq_weight, 
                    # 'm_cliq_ksize': m_cliq_ksize,
                    # 'm_cliq_kmax': m_cliq_kmax,
                    # 'm_pct_cliq_weight_one': m_pct_cliq_weight_one,
                    # 'm_max_sg': m_max_sg_cnt,
                    # 'm_max_sg_step': m_max_sg_step,
                }

    return run_metrics



In [ ]:
params = set_params()
experiment_path = create_experiment(process_run, params)


>>> INFO: START RUN00 with hparam = {'ncl': 10, 'nbk': 1, 'lr': 0.0005, 'dim': 16}


  0%|          | 0/9912422 [00:00<?, ?it/s]

  0%|          | 0/28881 [00:00<?, ?it/s]

  0%|          | 0/1648877 [00:00<?, ?it/s]

  0%|          | 0/4542 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
